In [ ]:
# Le date vengono ordinate in base al primo numero, ovvero al giorno, mese e anno non vengono cagati

In [1]:
import pandas as pd
import operator

In [2]:
# Importa i dati dal file excel
#Modifica la precisione nella visualizzazione delle cifre
pd.options.display.precision = 10

#Legge il csv e lo stampa
df_DatiSensori = pd.read_excel('Dati-gruppo1.xlsx')

In [ ]:
#Mostra tutte le colonne
# pd.set_option('display.max_rows', None)

df_DatiSensori

In [3]:
# Query 1

# query effettuata con un unico comando
# df_Benzene = df_DatiSensori.sort_values(by='C6H6_ug/m3', ascending =False)[['postazione', 'Data', 'C6H6_ug/m3']].head(100)

# creazione di un dataFrame ordinato in modo discendente per i valori di benzene
df_C6H6 = df_DatiSensori.sort_values(by='C6H6_ug/m3', ascending =False).drop_duplicates(subset=['Data'])
# creazione di un dataFrame contenente i 100 maggiori valori di benzene
query_C6H6 = df_C6H6[['postazione', 'Data', 'C6H6_ug/m3']]
query_C6H6.head(100)

,postazione,Data,C6H6_ug/m3
136674,ATM10_01481,2021-07-10 13:30:00,10.3999996185
113015,ATM07_01480,2021-11-08 09:55:00,5.9000000954
154856,ATM10_01481,2021-11-20 16:40:00,3.0000000000
154849,ATM10_01481,2021-11-20 16:05:00,2.7999999523
136677,ATM10_01481,2021-07-10 13:45:00,2.5000000000
...,...,...,...
189105,ATM14_01486,2021-11-06 14:45:00,0.8999999762
188606,ATM14_01486,2021-11-04 21:10:00,0.8999999762
189102,ATM14_01486,2021-11-06 14:30:00,0.8999999762
189101,ATM14_01486,2021-11-06 14:25:00,0.8999999762


In [ ]:
# Query 2 per il sensore H2S

# query effettuata con un unico comando
# df_H2S = df_DatiSensori.sort_values(by='H2S_ug/m3', ascending =False)[['postazione', 'Data', 'H2S_ug/m3']].head(100)
# creazione di un dataFrame ordinato in modo discendente per i valori di acido solfidrico del sensore H2S
df_H2S = df_DatiSensori.sort_values(by='H2S_ug/m3', ascending =False).drop_duplicates(subset=['Data'])
# creazione di un dataFrame contenente i 100 maggiori valori di acido solfidrico del sensore H2SJ
query_H2S = df_H2S[['postazione', 'Data', 'H2S_ug/m3', 'H2SJ_ug/m3']].head(100)
query_H2S

In [ ]:
# Query 2 per il sensore H2SJ

# query effettuata con un unico comando
# df_H2SJ = df_DatiSensori.sort_values(by='H2SJ_ug/m3', ascending =False)[['postazione', 'Data', 'H2SJ_ug/m3']].head(100)
# creazione di un dataFrame ordinato in modo discendente per i valori di acido solfidrico del sensore H2SJ
df_H2SJ = df_DatiSensori.sort_values(by='H2SJ_ug/m3', ascending =False).drop_duplicates(subset=['Data'])
# creazione di un dataFrame contenente i 100 maggiori valori di acido solfidrico del sensore H2SJ
query_H2SJ = df_H2SJ[['postazione', 'Data', 'H2SJ_ug/m3']].head(100)
query_H2SJ

In [ ]:
# Query 3 per il sensore VOC

# query effettuata con un unico comando
# df_VOC = df_DatiSensori.sort_values(by='VOC_ppm', ascending =True)[['postazione', 'Data', 'VOC_ppm']].head(100)
# creazione di un dataFrame ordinato in modo discendente per i valori del sensore VOC
df_VOC = df_DatiSensori.sort_values(by='VOC_ppm', ascending =True).drop_duplicates(subset=['Data'])
# creazione di un dataFrame contenente i 100 più bassi valori del sensore VOC
query_VOC = df_VOC[['postazione', 'Data', 'VOC_ppm']].head(100)
query_VOC

In [ ]:
# Query 3 per il sensore PIDVOC

# query effettuata con un unico comando
# df_PIDVOC = df_DatiSensori.sort_values(by='PIDVOC_ppb', ascending =True)[['postazione', 'Data', 'PIDVOC_ppb']].head(100)
# creazione di un dataFrame ordinato in modo discendente per i valori del sensore PIDVOC
df_PIDVOC = df_DatiSensori.sort_values(by='PIDVOC_ppb', ascending =True).drop_duplicates(subset=['Data'])
# creazione di un dataFrame contenente i 100 più bassi valori del sensore PIDVOC
query_PIDVOC = df_PIDVOC[['postazione', 'Data', 'PIDVOC_ppb']].head(100)
query_PIDVOC

In [ ]:
# Query 4

In [16]:
#Elimina i valori nulli e calcola la media oraria sul dataframe creato
df_C6H6 = df_DatiSensori[['Data','C6H6_ug/m3']].copy()
df_C6H6['Data'] = pd.to_datetime(df_C6H6['Data'])
df_C6H6 = df_C6H6[df_C6H6['C6H6_ug/m3'].notna()]
indexName = df_C6H6[(df_C6H6['C6H6_ug/m3']==0)].index
df_C6H6.drop(indexName, inplace = True)
df_C6H6 = df_C6H6[['Data', 'C6H6_ug/m3']].resample("H", label = 'right', on = 'Data').mean()

In [ ]:
# Stampa le 50 ore con i livelli di benzene più alti
df_C6H6Max = df_C6H6.sort_values('C6H6_ug/m3', ascending = False).head(50)
df_C6H6Max

In [ ]:
# Stampa le 50 ore con i livelli di benzene più bassi
df_C6H6Min = df_C6H6.sort_values('C6H6_ug/m3', ascending = True).head(50)
df_C6H6Min

In [ ]:
#50 ore con il pù basso livello medio di benzene
df_C6H6 = df_DatiSensori[['ora','C6H6_ug/m3']].copy()
df_ore = df_C6H6.groupby('ora').mean().sort_values(['C6H6_ug/m3'], ascending =True).head(50).reset_index()
df_ore[['ora']]

In [ ]:
# Query 5

In [ ]:
#Elimina i valori nulli e calcola la media oraria sul dataframe creato
df_AcidoSolf = df_DatiSensori[['Data','H2S_ug/m3','H2SJ_ug/m3' ]].copy()
df_AcidoSolf['Data'] = pd.to_datetime(df_AcidoSolf['Data'])
df_AcidoSolf = df_AcidoSolf[df_AcidoSolf['H2S_ug/m3'].notna()]
df_AcidoSolf = df_AcidoSolf[df_AcidoSolf['H2SJ_ug/m3'].notna()]
indexName = df_AcidoSolf[(df_AcidoSolf['H2S_ug/m3']==0)].index
indexName = df_AcidoSolf[(df_AcidoSolf['H2SJ_ug/m3']==0)].index
df_AcidoSolf.drop(indexName, inplace = True)
df_AcidoSolf = df_AcidoSolf[['Data', 'H2S_ug/m3', 'H2SJ_ug/m3']].resample("H", label = 'right', on = 'Data').mean()

In [ ]:
# Stampa le 50 ore con i livelli di acido solfidrico del sensore H2S più alti
df_H2SMax = df_AcidoSolf[df_AcidoSolf.columns[:-1]].sort_values('H2S_ug/m3', ascending = False).head(50)
df_H2SMax

,H2S_ug/m3
Data,
2021-08-11 18:00:00,92.9642855326
2021-08-11 19:00:00,90.9416669210
2021-07-31 19:00:00,87.6684212936
2021-08-11 20:00:00,71.9750002225
2021-07-31 20:00:00,69.6692310113
2021-07-01 20:00:00,66.9979173342
2021-06-29 18:00:00,66.0000003841
2021-07-28 20:00:00,64.5111114052
2021-07-10 18:00:00,64.1800003409


In [ ]:
# Stampa le 50 ore con i livelli di acido solfidrico del sensore H2S più basso
df_H2SMin = df_AcidoSolf[df_AcidoSolf.columns[:-1]].sort_values('H2S_ug/m3', ascending = True).head(50)
df_H2SMin

,H2S_ug/m3
Data,
2021-07-01 07:00:00,3.0454544804
2021-06-15 09:00:00,4.2727271919
2021-07-04 08:00:00,4.3194444709
2021-06-15 13:00:00,4.3282050467
2021-07-12 13:00:00,4.5200000383
2021-07-01 23:00:00,4.5466666500
2021-06-21 02:00:00,4.6111110979
2021-10-18 10:00:00,4.6249999166
2021-10-20 09:00:00,4.6346938464


In [ ]:
# Stampa le 50 ore con i livelli di acido solfidrico del sensore H2SJ più alti
df_H2SJMax = df_AcidoSolf['H2SJ_ug/m3']
df_H2SJMax= df_H2SJMax.to_frame().sort_values('H2SJ_ug/m3', ascending = False).head(50)
df_H2SJMax

,H2SJ_ug/m3
Data,
2021-06-29 00:00:00,27.3925924963
2021-10-18 05:00:00,24.9048773156
2021-06-24 21:00:00,24.0804883627
2021-10-18 04:00:00,23.9319141907
2021-06-22 15:00:00,23.8913049562
2021-08-13 23:00:00,23.3565211199
2021-07-02 10:00:00,22.6272728364
2021-07-14 11:00:00,21.8096153325
2021-07-26 15:00:00,21.0155551387


In [ ]:
# Stampa le 50 ore con i livelli di acido solfidrico del sensore H2SJ più bassi
df_H2SJMin = df_AcidoSolf['H2SJ_ug/m3']
df_H2SJMin= df_H2SJMin.to_frame().sort_values('H2SJ_ug/m3', ascending = True).head(50)
df_H2SJMin

,H2SJ_ug/m3
Data,
2021-06-15 03:00:00,0.2086956588
2021-11-05 12:00:00,0.2263157976
2021-06-14 19:00:00,0.2434782655
2021-11-05 11:00:00,0.2526315870
2021-06-15 01:00:00,0.2529411851
2021-11-05 02:00:00,0.2533333426
2021-11-05 09:00:00,0.2541666751
2021-06-15 05:00:00,0.2666666761
2021-11-05 01:00:00,0.2708333417


In [ ]:
# Query 6

In [9]:
#Elimina i valori nulli e calcola la media oraria sul dataframe creato
df_VOC = df_DatiSensori[['Data','VOC_ppm','PIDVOC_ppb' ]].copy()
df_VOC['Data'] = pd.to_datetime(df_VOC['Data'])
df_VOC = df_VOC[df_VOC['VOC_ppm'].notna()]
df_VOC = df_VOC[df_VOC['PIDVOC_ppb'].notna()]
indexName = df_VOC[(df_VOC['VOC_ppm']==0)].index
indexName = df_VOC[(df_VOC['PIDVOC_ppb']==0)].index
df_VOC.drop(indexName, inplace = True)
df_VOC = df_VOC[['Data', 'VOC_ppm', 'PIDVOC_ppb']].resample("H", label = 'right', on = 'Data').mean()

In [ ]:
# Stampa le 50 ore con i livelli di VOC del sensore VOC più alti
df_VOCMax = df_VOC[df_VOC.columns[:-1]].sort_values('VOC_ppm', ascending = False).head(50)
df_VOCMax

In [ ]:
# Stampa le 50 ore con i livelli di VOC del sensore VOC più bassi
df_VOCMin = df_VOC[df_VOC.columns[:-1]].sort_values('VOC_ppm', ascending = True).head(50)
df_VOCMin

,VOC_ppm
Data,
2021-07-10 20:00:00,0.0000000000
2021-06-04 05:00:00,0.0000000000
2021-06-04 04:00:00,0.0000000000
2021-07-14 08:00:00,0.0000000000
2021-11-05 02:00:00,0.0000000000
2021-06-04 01:00:00,0.0000000000
2021-11-05 09:00:00,0.0000000000
2021-06-04 03:00:00,0.0000000000
2021-11-05 03:00:00,0.0000000000


In [ ]:
# Stampa le 50 ore con i livelli di VOC del sensore PIDVOC più alti
df_PIDVOCMax = df_VOC['PIDVOC_ppb']
df_PIDVOCMax= df_PIDVOCMax.to_frame().sort_values('PIDVOC_ppb', ascending = False).head(50)
df_PIDVOCMax

,PIDVOC_ppb
Data,
2021-11-19 01:00:00,940.1627935587
2021-11-19 00:00:00,893.4657094683
2021-11-18 16:00:00,840.6181689176
2021-11-18 15:00:00,834.5936144971
2021-11-18 02:00:00,798.4999921799
2021-11-21 19:00:00,777.1927236730
2021-11-20 16:00:00,760.0241323833
2021-11-20 15:00:00,737.3847467859
2021-11-18 17:00:00,723.2585323148


In [ ]:
# Stampa le 50 ore con i livelli di VOC del sensore PIDVOC più bassi
df_PIDVOCMin = df_VOC['PIDVOC_ppb']
df_PIDVOCMin= df_PIDVOCMin.to_frame().sort_values('PIDVOC_ppb', ascending = True).head(50)
df_PIDVOCMin

In [ ]:
# Query 7

In [ ]:
#Elimina i valori nulli e calcola la media oraria sul dataframe creato
df_Fail = df_DatiSensori[['Data','TRS_ppb','VOC_ppm']].copy()
df_Fail['Data'] = pd.to_datetime(df_Fail['Data'])
indexName = df_Fail[(df_Fail['TRS_ppb']>=-1)].index
df_Fail.drop(indexName, inplace = True)
df_Fail = df_Fail[['Data','TRS_ppb','VOC_ppm']].resample("d", label = 'right', on = 'Data').count()
df_Fail

In [131]:
#Tre giornate con maggior numero fallimenti
df_C6H6 = df_DatiSensori[['Data','C6H6_ug/m3']].copy()
df_C6H6['Data'] = pd.to_datetime(df_C6H6['Data'].dt.strftime('%d/%m/%Y'))
df_C6H6['Giorno'] = df_C6H6['Data']
df_C6H6 = df_C6H6[df_C6H6['C6H6_ug/m3'].isna()]
df_C6H6 = df_C6H6[(df_C6H6['C6H6_ug/m3']!='0')]
df_C6H6 = df_C6H6.groupby('Giorno').count().sort_values('Data', ascending = False).head(3)
df_C6H6

,Data,C6H6_ug/m3
Giorno,,
2021-11-07,1440,0
2021-08-14,1282,0
2021-04-15,962,0


In [138]:
df_C6H6 = df_DatiSensori[['Data','C6H6_ug/m3']].copy()
df_C6H6['Data'] = pd.to_datetime(df_C6H6['Data'].dt.strftime('%d/%m/%Y'))
df_C6H6['Giorno'] = df_C6H6['Data']
df_C6H6 = df_C6H6[df_C6H6['C6H6_ug/m3'].isna()]
df_C6H6 = df_C6H6[(df_C6H6['C6H6_ug/m3']!='0')]
df_C6H6 = df_C6H6.groupby('Giorno').count().sort_values('Data', ascending = True).head(3)
df_C6H6

,Data,C6H6_ug/m3
Giorno,,
2021-10-20,5,0
2021-10-24,5,0
2021-08-07,6,0
2021-10-23,6,0
